In [1]:
import warnings
warnings.filterwarnings("ignore")  # "error", "ignore", "always", "default", "module" or "once"

In [2]:
import numpy as np
import pandas as pd
import random
import os
import copy
#from imutils import paths
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score as acc
from sklearn.metrics import precision_score as precision
from sklearn.metrics import recall_score as recall
from sklearn.metrics import f1_score as f1

from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from lightgbm import LGBMClassifier
from xgboost.sklearn import XGBClassifier

import keras
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Input, ReLU
from tensorflow.keras.models import Model

# Original Data

In [3]:
#declear path to your data
drebin_data_path = 'data/drebin.csv'
columns = list(pd.read_csv('data/dataset-features-categories.csv', header = None).iloc[:,0])
# Importing the dataset
Drebin_data = pd.read_csv(drebin_data_path, names = columns)

X = Drebin_data.iloc[:,range(0,Drebin_data.shape[1]-1)].values
y = Drebin_data.iloc[:, -1].values

lb = LabelBinarizer()
y = lb.fit_transform(y)

# Splitting the dataset into the Training set and Test set
X_train, X_test, y_train, y_test = train_test_split(X, y.astype(int), test_size = 0.3, random_state = 0)

# Feature Scaling
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Train-Test

In [4]:
classifiers = {"SVM": SVC(kernel = 'linear', degree=3), "MLP": MLPClassifier(random_state=1, max_iter=300), "XGB": XGBClassifier(), "LGBM": LGBMClassifier()} 

for classifier_pair in classifiers.items():
    print("---------------------------")
    print(classifier_pair[0])
    classifier = classifier_pair[1]
    classifier.fit(X_train, y_train)
    # Predicting the Test set results
    y_pred = classifier.predict(X_test)
    #compute accuracy_score
    accuracy = acc(y_test, y_pred)
    print('accuracy', accuracy)
    
print("---------------------------")

---------------------------
SVM
accuracy 0.9796054090002216
---------------------------
MLP
accuracy 0.9895810241631567
---------------------------
XGB
accuracy 0.9904677455109732
---------------------------
LGBM
accuracy 0.9904677455109732
---------------------------


# AutoKeras Model

In [5]:
saved_model=keras.models.load_model('structured_data_classifier/best_model', compile=True)
print(saved_model.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 215)]             0         
                                                                 
 multi_category_encoding (Mu  (None, 215)              0         
 ltiCategoryEncoding)                                            
                                                                 
 normalization (Normalizatio  (None, 215)              431       
 n)                                                              
                                                                 
 dense (Dense)               (None, 256)               55296     
                                                                 
 re_lu (ReLU)                (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 32)                8224  

In [4]:
def autoKerasModel(path_best_model):
    saved_model = keras.models.load_model(path_best_model, compile=True)
    input_layer = Input(shape=(215,))
    x = saved_model.layers[1](input_layer)
    x = saved_model.layers[2](x)
    x = Dense(units=256)(x)
    x = ReLU()(x)
    x = Dense(units=32)(x)
    x = ReLU()(x)
    x = Dense(units=1)(x)
    x = saved_model.layers[-1](x)
    new_model = Model(inputs=input_layer, outputs=x)
    return new_model

In [7]:
autokeras_model=autoKerasModel('structured_data_classifier/best_model')
autokeras_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

autokeras_model.fit(X_train, y_train, epochs=15)
results = autokeras_model.evaluate(X_test, y_test)

print("test loss, test acc:", results)
print(autokeras_model.summary())

Epoch 1/15
329/329 [==============================] - 5s 7ms/step - loss: 0.0933 - accuracy: 0.9643
Epoch 2/15
329/329 [==============================] - 2s 6ms/step - loss: 0.0414 - accuracy: 0.9853
Epoch 3/15
329/329 [==============================] - 2s 6ms/step - loss: 0.0297 - accuracy: 0.9905
Epoch 4/15
329/329 [==============================] - 2s 6ms/step - loss: 0.0221 - accuracy: 0.9934
Epoch 5/15
329/329 [==============================] - 2s 6ms/step - loss: 0.0177 - accuracy: 0.9943
Epoch 6/15
329/329 [==============================] - 2s 6ms/step - loss: 0.0129 - accuracy: 0.9959
Epoch 7/15
329/329 [==============================] - 2s 6ms/step - loss: 0.0120 - accuracy: 0.9971
Epoch 8/15
329/329 [==============================] - 2s 6ms/step - loss: 0.0110 - accuracy: 0.9973
Epoch 9/15
329/329 [==============================] - 2s 6ms/step - loss: 0.0130 - accuracy: 0.9962
Epoch 10/15
329/329 [==============================] - 2s 6ms/step - loss: 0.0260 - accuracy: 0.9927

# Ensemble Learning - Majority Voting

In [5]:
def majority_voting(y_preds, y_test):
    assert y_preds.shape[0] == len(y_test), "y_preds's length is: {} while y_test's length is: {}. They should be equal.".format(y_preds.shape[0],len(y_test))
    y_pred_vote = []
    for preds in y_preds:
        if sum(preds) >= 3:
            y_pred_vote.append(1)
        else:
            y_pred_vote.append(0)
    #compute accuracy_score
    accuracy = acc(y_test, y_pred_vote)
    return accuracy

In [9]:
y_preds = np.ndarray(shape=(5,len(y_test)))
i=0
for classifier_pair in classifiers.items():
    classifier = classifier_pair[1]
    y_preds[i] = classifier.predict(X_test)
    i += 1
y_preds[i] = np.transpose(autokeras_model.predict(X_test, verbose=0))
y_preds = np.transpose(y_preds)
print('accuracy', majority_voting(y_preds, y_test))

accuracy 0.9909111061848814


# Label Flipping

In [28]:
def attack_label_flipping(X_train, X_test, y_train, y_test, per, classifier_name, classifier):
    flipped_data = specific_label_flipping(y_train, per, 1)
    accuracy = 0
    if classifier_name == "AutoKeras":
        classifier.fit(X_train, flipped_data, verbose=0, epochs=15)
        y_pred = np.transpose(classifier.predict(X_test, verbose=0))
        accuracy = classifier.evaluate(X_test, y_test, verbose=0)[1]
    else:
        classifier.fit(X_train, flipped_data)
        y_pred = classifier.predict(X_test)
        accuracy = acc(y_test, y_pred)
    return y_pred, accuracy

In [22]:
#Flipping Random
def random_label_flipping(y_train, per):
    flip_count = int(per*(len(y_train)))
    flipped_data = copy.deepcopy(y_train)
    indices = random.sample(range(len(flipped_data)), flip_count)
    for j in indices:
        flipped_data[j] = (flipped_data[j] + 1)%2
    return flipped_data

#Flipping Specific
def specific_label_flipping(y_train, per, target):
    flipped_data = copy.deepcopy(y_train)
    possible_indices = []
    for i in range(len(y_train)):
        if y_train[i] == target:
            possible_indices.append(i)
    flip_count = int(per*(len(possible_indices)))
    indices = random.sample(possible_indices, flip_count)
    for j in indices:
        flipped_data[j] = (flipped_data[j] + 1)%2
    return flipped_data

In [32]:
percentages = [0.01, 0.05, 0.1, 0.2]
per = percentages[3]
poisoned_accuracies = {"SVM": [], "MLP": [], "XGB": [], "LGBM": [], "AutoKeras": []}
ensemble_accuracies = []

print("---------------------------")
for i in range(5):
    print("Trial #{} is starting...".format(i+1))
    
    poisoned_classifiers = {"SVM": SVC(kernel = 'linear', degree=3), "MLP": MLPClassifier(random_state=1, max_iter=300), "XGB": XGBClassifier(), "LGBM": LGBMClassifier()} 
    poisoned_autokeras_model=autoKerasModel('structured_data_classifier/best_model')
    poisoned_autokeras_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    poisoned_classifiers["AutoKeras"] = poisoned_autokeras_model
    
    y_preds = np.ndarray(shape=(len(poisoned_classifiers),len(y_test)))
    j=0
    for classifier_pair in poisoned_classifiers.items():
        poisoned_y_pred, poisoned_accuracy = attack_label_flipping(X_train, X_test, y_train, y_test, per, classifier_pair[0], classifier_pair[1])
        y_preds[j] = poisoned_y_pred
        j+=1
        poisoned_accuracies[classifier_pair[0]].append(poisoned_accuracy)
    y_preds = np.transpose(y_preds)
    
    ensemble_accuracy = majority_voting(y_preds, y_test)
    ensemble_accuracies.append(ensemble_accuracy)
    
    print("Trial #{} is completed with accuracy: {}".format(i+1, ensemble_accuracy))
    print("---------------------------")

print("Random Poisoning - {}".format(per))
for classifier_pair in poisoned_accuracies.items():
    accuracies = classifier_pair[1]
    print("{}'s average accuracy: {}".format(classifier_pair[0], sum(accuracies)/len(accuracies)))
print("Average ensemble accuracy:", sum(ensemble_accuracies)/len(ensemble_accuracies))

---------------------------
Trial #1 is starting...
Trial #1 is completed with accuracy: 0.97251163821769
---------------------------
Trial #2 is starting...
Trial #2 is completed with accuracy: 0.97628020394591
---------------------------
Trial #3 is starting...
Trial #3 is completed with accuracy: 0.9711815561959655
---------------------------
Trial #4 is starting...
Trial #4 is completed with accuracy: 0.9729549988915983
---------------------------
Trial #5 is starting...
Trial #5 is completed with accuracy: 0.9771669252937264
---------------------------
Random Poisoning - 0.2
SVM's average accuracy: 0.9626246951895366
MLP's average accuracy: 0.9632454001330082
XGB's average accuracy: 0.9724673021502992
LGBM's average accuracy: 0.976989581024163
AutoKeras's average accuracy: 0.9781866669654846
Average ensemble accuracy: 0.9740190645089781


# SECML Poisoning

In [4]:
from secml.ml.classifiers.sklearn.c_classifier_logistic import CClassifierLogistic
from secml.adv.attacks.poisoning.c_attack_poisoning_logistic_regression import CAttackPoisoningLogisticRegression
from secml.data.c_dataset import CDataset
from secml.ml.peval.metrics import CMetricAccuracy

lob, ub = 0., 1.  # Bounds of the attack space. Can be set to `None` for unbounded

# Should be chosen depending on the optimization problem
solver_params = {
    'eta': 0.25,
    'eta_min': 2.0,
    'eta_max': None,
    'max_iter': 100,
    'eps': 1e-6
}

dataset = CDataset(X_train, y_train)
metric = CMetricAccuracy()

# train SVM in the dual space, on a linear kernel, as needed for poisoning
c_classifier_LR = CClassifierLogistic()
print("Training of c_classifier_LR...")
c_classifier_LR.fit(X_train, y_train)
print("Training of c_classifier_LR is done!")

Training of c_classifier_LR...
Training of c_classifier_LR is done!


In [5]:
poisoning_strength = 0.1 # Percentage of points to poison in training data
n_poisoning_points = 2  # Number of poisoning points to generate

pois_attack = CAttackPoisoningLogisticRegression(classifier=c_classifier_LR,
                                training_data=dataset,
                                val=dataset,
                                lb=lob, 
                                ub=ub,
                                solver_params=solver_params)
pois_attack.n_points = n_poisoning_points

# Run the poisoning attack
print("Attack started...")
pois_y_pred, _, pois_points_ds, _ = pois_attack.run(X_test, y_test)
print("Attack complete!")

Attack started...
Attack complete!


# Reading Poisoned data & adding to/replacing with training dataset

In [20]:
#declear path to your data
pois_points_path = 'poisoning-20.csv'
# Importing the dataset
pois_points_ds = pd.read_csv(pois_points_path)
pois_points_X = pois_points_ds.iloc[:,range(0,pois_points_ds.shape[1]-1)].values
pois_points_y = pois_points_ds.iloc[:, -1].values

pois_points_y = lb.fit_transform(pois_points_y)

In [21]:
poisoned_accuracies = {"SVM": [], "MLP": [], "XGB": [], "LGBM": [], "AutoKeras": []}
ensemble_accuracies = []
print("---------------------------")
for trial in range(5):
    pois_X_train = copy.deepcopy(X_train)
    pois_y_train = copy.deepcopy(y_train)

    #Uncomment below to RANDOMLY REPLACE with poisoned data
    to_be_replaced = random.sample(range(len(pois_X_train)), len(pois_points_X))
    new_data = 0
    for index in to_be_replaced:
        pois_X_train[index] = pois_points_X[new_data]
        pois_y_train[index] = pois_points_y[new_data]
        new_data += 1

    #Uncomment below to ADD poisoned data
    """pois_X_train = np.append(pois_X_train, pois_points_X, axis=0)
    pois_y_train = np.append(pois_y_train, pois_points_y)"""

    print("Trial #{} is starting...".format(trial+1))
    
    classifiers_secml = {"SVM": SVC(kernel = 'linear', degree=3), "MLP": MLPClassifier(random_state=1, max_iter=300), "XGB": XGBClassifier(), "LGBM": LGBMClassifier()} 
    
    y_preds = np.ndarray(shape=(5,len(y_test)))
    j=0
    for classifier_pair in classifiers_secml.items():
        classifier = classifier_pair[1]
        classifier.fit(pois_X_train, pois_y_train)
        # Predicting the Test set results
        poisoned_y_pred = classifier.predict(X_test)
        y_preds[j] = poisoned_y_pred
        #compute accuracy_score
        poisoned_accuracy = acc(y_test, poisoned_y_pred)
        poisoned_accuracies[classifier_pair[0]].append(poisoned_accuracy)
        j+=1
        
    ak_model_secml=autoKerasModel('structured_data_classifier/best_model')
    ak_model_secml.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    ak_model_secml.fit(pois_X_train, pois_y_train, epochs=15, verbose=0)
    y_preds[j] = np.transpose(ak_model_secml.predict(X_test, verbose=0))
    poisoned_accuracies["AutoKeras"].append(ak_model_secml.evaluate(X_test, y_test, verbose=0)[1])

    y_preds = np.transpose(y_preds)
    
    ensemble_accuracy = majority_voting(y_preds, y_test)
    ensemble_accuracies.append(ensemble_accuracy)
    
    print("Trial #{} is completed with accuracy: {}".format(trial+1, ensemble_accuracy))
    print("---------------------------")

print("SECML Poisoning - 20%")
for classifier_pair in poisoned_accuracies.items():
    accuracies = classifier_pair[1]
    print("{}'s average accuracy: {}".format(classifier_pair[0], sum(accuracies)/len(accuracies)))
print("Average ensemble accuracy:", sum(ensemble_accuracies)/len(ensemble_accuracies))

---------------------------
Trial #1 is starting...
Trial #1 is completed with accuracy: 0.9893593438262026
---------------------------
SECML Poisoning - 10%
SVM's average accuracy: 0.9687430724894702
MLP's average accuracy: 0.9824872533806251
XGB's average accuracy: 0.9886943028153403
LGBM's average accuracy: 0.9895810241631567
AutoKeras's average accuracy: 0.9895810484886169
Average ensemble accuracy: 0.9893593438262026
